### Initialize GPU configuration

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#config.log_device_placement = True 
# config.intra_op_parallelism_threads=16
# config.inter_op_parallelism_threads=16
sess = tf.Session(config=config)

from keras import backend as K
K.set_session(sess)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

### Initialize some arguments

In [ ]:
# root_dir = insert directory of the dataset images
# output_dir = insert where you want the models to be saved

image_size = 128

### Data Prep

In [ ]:
import pandas as pd

cols = ["Image Path", "Category"]
train = pd.read_csv('train-datagen.csv', header=None, names=cols)

In [ ]:
val = pd.read_csv('val-datagen.csv', header=None, names=cols)
val

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
batch_size = 64

train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# this is the augmentation configuration we will use for validation:
# only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical')  # since we use categorical_crossentropy loss

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=val,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

In [ ]:
#confirm the scaling works

batchX, batchY = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

## Model Prep

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
base_model.summary()

#### Freeze all the layers 

In [ ]:
# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in base_model.layers:
    print(layer, layer.trainable)


In [ ]:
from keras import models
from keras import layers
from keras import optimizers


# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(base_model)

# Add new layers
model.add(layers.Flatten())
model.add(layers.BatchNormalization())

model.add(layers.Dense(512, activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))

model.add(layers.Dense(128, activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(64, activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(30, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

## Train the model

In [ ]:
# Compile the model
import keras
from keras import backend as K
import tensorflow as tf
import functools 

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc', top3_acc])


In [ ]:
# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size ,
    epochs=200,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1,
    max_queue_size=20000,
    workers=4)

# Save the model
model.save(f'{output_dir}\\inceptionresnet.h5') 

## See performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Predict 

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a generator for prediction
test_generator = test_datagen.flow_from_directory(
        f'{root_dir}\\test',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)


In [ ]:
import numpy as np 

# Get the filenames from the generator
fnames = test_generator.filenames
test_dir = f'{root_dir}\\test'
# Get the ground truth from generator
ground_truth = test_generator.classes

# Get the label to class mapping from the generator
label2index = test_generator.class_indices

# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())

# Get the test loss and accuracy from the model
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples/test_generator.batch_size,verbose=1)
print("Testing Loss = " + str(test_loss))
print("Testing Accuracy = " + str(test_acc))
print("\n")
# Get the predictions from the model using the generator
predictions = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size,verbose=1)



predicted_classes = np.argmax(predictions,axis=1)

errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),test_generator.samples))

# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
    
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
    
    original = load_img('{}/{}'.format(test_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

## Results:

### Validation Accuracy : 0.0794 
### Accuracy : 0.1520

